In [40]:
from dolfin import *
import numpy as np
import pylab as plt
import os
import datetime

def wave_eq(the_model                 , 
            num_intervals = 10        ,                                     
            out_name      = 'soln.pvd',
            build_dir     = ''):
    if( build_dir != '' ):
        if( os.path.exists(build_dir) == False ):
            os.system('mkdir %s'%(build_dir))
    
    #pass to local variable names
    c     = the_model.c
    mesh  = the_model.mesh
    dt    = the_model.dt
    T_0   = the_model.T_0
    T_1   = the_model.T_1
    delta = the_model.src
    V     = the_model.V
    N     = the_model.N
    
    # Time variables
    t = T_0
    T = T_1 - T_0

    #interval for showing plots
    plot_interval = np.floor(T / (dt * num_intervals))
    
    # Previous and current solution
    u1= interpolate(Constant(0.0), V)
    u0= interpolate(Constant(0.0), V)

    # Variational problem at each time
    u = TrialFunction(V)
    v = TestFunction(V)

    a = u*v*dx + dt*dt*c*c*inner(grad(u), grad(v))*dx
    L = 2*u1*v*dx-u0*v*dx

    bc = DirichletBC(V, 2.0, "on_boundary")
    A, b = assemble_system(a, L, bc)

    i_plot = 0
    u=Function(V)
    up=[]
    fid = File('solution.pvd')
    while t <= T_1:
        A, b = assemble_system(a, L, bc)
        delta_eval = delta(t)
        delta_eval.apply(b)
        solve(A, u.vector(), b)
        u0.assign(u1)
        u1.assign(u)
        t += dt

        # Reduce the range of the solution so that we can see the waves
        j = 0
        min_signal = 0.01
        for i in u.vector():
            i = min(.01, i)
            i = max(-.01, i)
            u.vector()[j] = i;
            j += 1

        if( (i_plot % plot_interval) == 0 ):
            #print('Hello about to plot...')
            print('About to plot')
            up.append(Function(V))
            up[len(up) - 1].assign(u)
            #plot(up, interactive=False, title=str('t = ' + str(t)))
            #plt.plot(mesh.coordinates(), u.vector())
            #fig.write_png(str('u-' + t + '.png'))
            
            
        i_plot += 1
    fid = File('%s%s'%(build_dir,out_name))
    num_plots = num_intervals
    for i in range(0,len(up)):
        t = dt * ( 1 + i * (T_1-T_0) / (dt * num_plots) )
        fid << up[i], t

In [41]:
class WaveModel:
    def __init__(self, mesh, \
                 space='Lagrange',     \
                 u0='0.0',              \
                 u1='0.0',              \
                 deg=1,                 \
                 c='1.0',       \
                 gamma_1='1.0', \
                 gamma_2='1.0', \
                 T_0=0,                 \
                 T_1=1,                 \
                 num_steps=100):
        print('about to construct')
        self.mesh     = mesh
        self.V        = FunctionSpace(mesh, space, deg)
        self.V_v      = VectorFunctionSpace(mesh, space, deg, dim=2)
        self.deg      = deg
        self.c        = Expression(c, domain=mesh, degree=1)
        self.u        = Expression(u0, domain=mesh, degree=1)
        self.u0       = Expression(u0, domain=mesh, degree=1)
        self.u0_deriv = Expression(u1, domain=mesh, degree=1)
        self.gamma_1  = Expression(gamma_1, domain=mesh, degree=1)
        self.gamma_2  = Expression(gamma_2, domain=mesh, degree=1)
        self.T_0      = T_0
        self.T_1      = T_1
        self.N        = num_steps
        self.dt       = (T_1 - T_0) / (num_steps-1)
        self.src      = lambda t : Constant(0.0)
    
    #source must be passed as a function of time, i.e. of type (t) -> ( R^2 -> R )
    def set_source(self, the_source):
        self.src = the_source
        
    #printing function for debugging
    def pretty_print(self):
        print('mesh = %s'%(self.mesh))
        print('space = %s'%(self.V))
        print('c = %s'%(self.c))
        print('T_0 = %s'%(self.T_0))
        print('T_1 = %s'%(self.T_1))
        print('N = %s'%(self.N))
        print('dt = %s'%(self.dt))
        print('source = %s'%(self.src))
        print('u0 = %s'%(self.u0))
        print('u0_deriv = %s'%(self.u0_deriv))
        
    #TODO: Modify so that the build_directory is passed as an argument, not dynamically calculated
    def go_forward(self,
                   num_intervals = 10        ,                                     
                   out_name      = 'soln',
                   build_dir     = ''):

    
        #pass to local variable names
        c        = self.c
        mesh     = self.mesh
        V        = self.V
        u_prev   = self.u0
        u0       = self.u0
        u0_deriv = self.u0_deriv
        gamma_1  = self.gamma_1
        gamma_2  = self.gamma_2
        dt       = self.dt
        T_0      = self.T_0
        T_1      = self.T_1
        delta    = self.src
        N        = self.N
        
        # Time variables
        t = T_0
        T = T_1 - T_0
        
        u = u_prev
        
        u_grad = grad(u)
        u_grad_x = u_grad[0]
        u_grad_y = u_grad[1]

        #interval for showing plots
        plot_interval = np.floor(T / (dt * num_intervals))

        #auxiliary variable initial advancement
        def getPhi(i):
            phi1 = TrialFunction(self.V)
            v1   = TestFunction(self.V)
        
            alpha = (-1) ** (i)
            
            a = phi1 * v1 * dx
            L = alpha * dt * c * (gamma_2-gamma_1) * grad(u)[i] * v1 * dx
        
            bc = DirichletBC(V,0,"on_boundary")
            A,b = assemble_system(a,L,bc)
        
            phi1_tilde = Function(self.V)
            solve(A,phi1_tilde.vector(),b)
            return phi1_tilde
    
        phi0_prev = Expression('0.0', domain=mesh, degree=self.deg)
        phi1_prev = Expression('0.0', domain=mesh, degree=self.deg)
        
        phi0 = getPhi(0)
        phi1 = getPhi(1)
        
        #setup computations
        u_prev = u0
        u = Function(self.V)
        u.assign = dt * u0_deriv + u0
        
        u_trial = TrialFunction(self.V)
        u_test  = TestFunction(self.V)
        
        A_u   = (1 / (dt**2) + (gamma_1 + gamma_2) / (2 * dt)) * u_trial * u_test * dx
        A_phi = u_trial * u_test * dx
        
        def phi_ind(i):
            if( i == 0 ):
                return phi0
            else:
                return phi1
            
        def lin_form_u():
            grad_phi0 = grad(phi0)
            grad_phi1 = grad(phi1)
            div_phi   = grad_phi0[0] + grad_phi1[1]
            curr_term = (2 / (dt**2) - gamma_1 * gamma_2) * u
            prev_term = ((gamma_1+gamma_2) / (2 * dt) - 1 / (dt**2)) * u_prev
            L = (curr_term + prev_term + div_phi) * u_test * dx
            return L
        
        def lin_form_phi(i):
            grad_u = grad(u)
            if(i == 0):
                prev_term      = phi0_prev
                curr_phi_term  = -dt * gamma_1 * phi0
                curr_grad_term = 2 * dt * c * c * (gamma_2 - gamma_1) * grad_u[0]
                L = (prev_term + curr_phi_term + curr_grad_term) * u_test * dx
                return L
            if(i == 1):
                prev_term      = phi1_prev
                curr_phi_term  = -dt * gamma_2 * phi1
                curr_grad_term = -2 * dt * c * c * (gamma_2 - gamma_1) * grad_u[1]
                L = (prev_term + curr_phi_term + curr_grad_term) * u_test * dx
                return L
            
        def step_u(tt):
            #MAKE SURE THIS IS A DEEP COPY FOR DEBUGGING PURPOSES!!! WOULD MESS
            #EVEYRTHING UP IF IT IS ONLY A SHALLOW COPY!!!
            u_prev_tmp = Function(V)
            u_prev_tmp = u
            L_u        = lin_form_u()
            bc_u       = DirichletBC(V,0,"on_boundary")
            A, b       = assemble_system(A_u,L_u,bc_u)
            delta_eval = self.src(tt)
            delta_eval.apply(b)
            solve(A, u.vector(), b)
            u_prev = u_prev_tmp
        
        def step_phi(i):
            L_phi = lin_form_phi(i)
            bc_phi       = DirichletBC(V,0,"on_boundary")
            A, b         = assemble_system(A_phi,L_phi,bc_phi)
            
            if( i == 0 ):
                phi_prev_tmp = phi0
                solve(A, phi0.vector(), b)
                phi0_prev = phi_prev_tmp
            else:
                phi_prev_tmp = phi1
                solve(A, phi1.vector(), b)
                phi1_prev = phi_prev_tmp        
        
        i_plot = 0
        up = []
        while t <= T_1:
            step_u(t)
            step_phi(0)
            step_phi(1)
            
            t += dt 
            
            if( (i_plot % plot_interval) == 0 ):
                print('About to plot')
                up.append(Function(V))
                up[len(up) - 1].assign(u)
                
            i_plot += 1
        x = datetime.datetime.now()
        curr_time = '%s%s-%s:%s'%(x.strftime("%B"),x.strftime("%d"), \
                                  x.strftime("%H"), \
                                  x.strftime("%M"))
        build_dir = '%s/%s'%(build_dir, curr_time)
        os.system('mkdir %s'%(build_dir))
        #fid = File('%s/%s'%(build_dir,out_name))
        fid = File("%s.pvd"%(out_name))
        num_plots = num_intervals
        print(len(up))
        for i in range(0,len(up)):
            t = dt * ( 1 + i * (T_1-T_0) / (dt * num_plots) )
            fid << up[i], t 
        os.system("sed -i \"\" \'s/UInt32/Int32/g\' %s*.vtu"%(out_name))
        os.system("sed -i \"\" \'s/f_[0-9]*/f/g\' %s*.vtu"%(out_name))
        os.system("mv %s* %s"%(out_name, build_dir))

In [42]:
def PML_expr_left(C,eta,a,i):
    eta_inv  = 1 / eta
    ref = eta + a
    C_over_eta = C / eta
    
    var = 'x[%s]'%(i)
    
    mon = '(%s - %s) * %s'%(var, ref, eta_inv)
    
    expr = '%s * (%s) * (%s)'% \
                 (C_over_eta, mon, mon)
    
    return '%s < %s ? (%s) : 0.0'%(var,ref,expr)

def PML_expr_right(C,eta,b,i):
    eta_inv  = 1 / eta
    ref = b - eta
    C_over_eta = C / eta
    
    var = 'x[%s]'%(i)
    
    mon = '(%s - %s) * %s'%(var, ref, eta_inv)
    
    expr = '%s * (%s) * (%s)'% \
                 (C_over_eta, mon, mon)
    
    return '%s > %s ? (%s) : 0.0'%(var,ref,expr)

def PML_expr_full(C,eta,a,b,i):
    left_expr  = PML_expr_left(C,eta,a,i)
    right_expr = PML_expr_right(C,eta,b,i)
    return '(%s) + (%s)'%(left_expr,right_expr)

plot(PML_expr_left(1.0,0.1,0.0,0))

RuntimeError: Don't know how to plot given object:
  x[0] < 0.1 ? (10.0 * ((x[0] - 0.1) * 10.0) * ((x[0] - 0.1) * 10.0)) : 0.0
and projection failed:
  'str' object has no attribute 'ufl_domain'

In [70]:
mesh      = RectangleMesh(Point(-2, -2), Point(2, 2), 25,25)
space     = "Lagrange"
deg       = 1
u0        = '0.0'
u0_deriv  = '0.0'
deg       = 1
c         = '1.0'
gamma_1   = PML_expr_full(1.0,0.2,-2.0,2.0,0)
gamma_2   = PML_expr_full(1.0,0.2,-2.0,2.0,1)
T_0       = 0
T_1       = 2
N         = 1000

the_model = WaveModel(mesh,     \
                      space,    \
                      u0,       \
                      u0_deriv, \
                      deg,      \
                      c,        \
                      gamma_1,  \
                      gamma_2,  \
                      T_0,      \
                      T_1,      \
                      N)

the_model.set_source(lambda t : PointSource(the_model.V,Point(0.0,0.0), 1.0/(t+1))
the_model.pretty_print()
the_model.go_forward(num_intervals=100, build_dir='figures')

SyntaxError: invalid syntax (<ipython-input-70-aa358ea6aefa>, line 27)

In [44]:
V = FunctionSpace(the_model.mesh, "Lagrange", 1)
Q = VectorFunctionSpace(the_model.mesh, "Lagrange", 1)

In [45]:
v = Function(V)
q = Function(Q)
z = v * q
print(type(v))
print(type(q))
print(type(z))

<class 'dolfin.function.function.Function'>
<class 'dolfin.function.function.Function'>
<class 'ufl.tensors.ComponentTensor'>


In [46]:
the_model.pretty_print()

mesh = <dolfin.cpp.generation.RectangleMesh object at 0x12bbc4c70>
space = FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 4128), FiniteElement('Lagrange', triangle, 1))
c = f_4135
T_0 = 0
T_1 = 1
N = 11
dt = 0.1
source = <function <lambda> at 0x12b3190d0>
u0 = f_4137
u0_deriv = f_4138


In [47]:
the_model.src

<function __main__.<lambda>(t)>

In [84]:
from dolfin import *
import pylab as plt
import numpy as np

# Create mesh and define function space
mesh = UnitSquareMesh(128, 128)
V = FunctionSpace(mesh, 'Lagrange', 1)
f = Expression("%s + %s"%(PML_expr_full(0.5,0.3,0.0,1.0,0), PML_expr_full(0.5,0.3,0.0,1.0,1)), degree=1)

ff = PointSource(V, Point(0.0,0.0),1.0)

c = plot(interpolate(f, V), mode='color')
plt.colorbar(c)

plt.figure()
c = plot(interpolate(f, V), mode='color', vmin=-3, vmax=3)
plt.colorbar(c)

plt.show()

AttributeError: 'dolfin.cpp.fem.PointSource' object has no attribute '_cpp_object'

[4.     4.     3.9204 ... 3.9204 4.     4.    ]


RuntimeError: Don't know how to plot given object:
  [4.     4.     3.9204 ... 3.9204 4.     4.    ]
and projection failed:
  'numpy.ndarray' object has no attribute 'ufl_domain'

Calling FFC just-in-time (JIT) compiler, this may take some time.


Invalid ranks 1 and 1 in product.


UFLException: Invalid ranks 1 and 1 in product.

In [78]:
psrc = lambda t : PointSource(the_model.V,Point(0.0,0.0), 1.0/(t+1))
plot(Expression("sin(x[0])", mesh=mesh, degree=1))

RuntimeError: Unable to cast Python instance to C++ type (compile in debug mode for details)